# SAS 데이터를 CSV로 변환 및 EDA

이 노트북은 SAS 파일(.sas7bdat)을 읽어서 CSV로 저장하고 기본적인 탐색적 데이터 분석(EDA)을 수행합니다.

## 1. 필요한 라이브러리 설치 및 임포트

In [ ]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
# !pip install pandas pyreadstat matplotlib seaborn numpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 한글 폰트 설정 (선택사항)
plt.rc('font', family='Malgun Gothic')  # Windows
# plt.rc('font', family='AppleGothic')  # Mac
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

## 2. SAS 파일 읽기 및 CSV로 저장

In [ ]:
# SAS 파일 경로 지정
sas_file_path = 'your_file.sas7bdat'  # 여기에 실제 파일 경로를 입력하세요

# SAS 파일 읽기
try:
    df = pd.read_sas(sas_file_path, encoding='utf-8')
    print(f"✓ SAS 파일 로드 성공!")
    print(f"데이터 크기: {df.shape[0]}행 × {df.shape[1]}열")
except Exception as e:
    print(f"✗ 오류 발생: {e}")
    print("\n다른 인코딩으로 시도 중...")
    try:
        df = pd.read_sas(sas_file_path, encoding='latin1')
        print("✓ SAS 파일 로드 성공! (latin1 인코딩)")
    except Exception as e2:
        print(f"✗ 다시 오류 발생: {e2}")

In [ ]:
# CSV 파일로 저장
csv_file_path = 'converted_data.csv'  # 저장할 CSV 파일명

df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')  # utf-8-sig는 엑셀에서 한글 깨짐 방지
print(f"✓ CSV 파일 저장 완료: {csv_file_path}")

## 3. 기본 정보 확인 (EDA)

In [ ]:
# 데이터 미리보기
print("=== 데이터 미리보기 (처음 5행) ===")
display(df.head())

In [ ]:
# 데이터 기본 정보
print("=== 데이터 기본 정보 ===")
print(df.info())

In [ ]:
# 컬럼명 확인
print("=== 컬럼 목록 ===")
print(df.columns.tolist())

In [ ]:
# 기술통계량
print("=== 수치형 변수 기술통계량 ===")
display(df.describe())

## 4. 결측치 확인

In [ ]:
# 결측치 개수 및 비율
print("=== 결측치 현황 ===")
missing_df = pd.DataFrame({
    '결측치 개수': df.isnull().sum(),
    '결측치 비율(%)': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_df = missing_df[missing_df['결측치 개수'] > 0].sort_values('결측치 개수', ascending=False)
display(missing_df)

In [ ]:
# 결측치 시각화
if len(missing_df) > 0:
    plt.figure(figsize=(10, 6))
    missing_df['결측치 비율(%)'].plot(kind='barh')
    plt.xlabel('결측치 비율 (%)')
    plt.title('변수별 결측치 비율')
    plt.tight_layout()
    plt.show()
else:
    print("결측치가 없습니다!")

## 5. 수치형 변수 분포 확인

In [ ]:
# 수치형 변수 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"수치형 변수 개수: {len(numeric_cols)}")
print(numeric_cols)

In [ ]:
# 수치형 변수 히스토그램 (상위 6개만)
if len(numeric_cols) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, col in enumerate(numeric_cols[:6]):
        df[col].hist(bins=30, ax=axes[i], edgecolor='black')
        axes[i].set_title(f'{col} 분포')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('빈도')
    
    plt.tight_layout()
    plt.show()
else:
    print("수치형 변수가 없습니다.")

## 6. 범주형 변수 확인

In [ ]:
# 범주형 변수 선택
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"범주형 변수 개수: {len(categorical_cols)}")
print(categorical_cols)

In [ ]:
# 범주형 변수의 고유값 개수
if len(categorical_cols) > 0:
    print("=== 범주형 변수의 고유값 개수 ===")
    for col in categorical_cols:
        print(f"{col}: {df[col].nunique()}개")

In [ ]:
# 범주형 변수 빈도 (첫 번째 변수만)
if len(categorical_cols) > 0:
    first_cat = categorical_cols[0]
    print(f"=== {first_cat} 빈도표 ===")
    value_counts = df[first_cat].value_counts()
    print(value_counts)
    
    # 시각화 (상위 10개만)
    plt.figure(figsize=(10, 6))
    value_counts.head(10).plot(kind='bar')
    plt.title(f'{first_cat} 상위 10개 범주 빈도')
    plt.xlabel(first_cat)
    plt.ylabel('빈도')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 7. 상관관계 분석 (수치형 변수)

In [ ]:
# 상관계수 행렬
if len(numeric_cols) > 1:
    correlation_matrix = df[numeric_cols].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                fmt='.2f', square=True, linewidths=1)
    plt.title('변수 간 상관관계 히트맵')
    plt.tight_layout()
    plt.show()
else:
    print("상관관계 분석을 위한 수치형 변수가 부족합니다.")

## 8. 데이터 요약 리포트

In [ ]:
print("="*50)
print("데이터 요약 리포트")
print("="*50)
print(f"총 행 수: {df.shape[0]:,}")
print(f"총 열 수: {df.shape[1]}")
print(f"수치형 변수: {len(numeric_cols)}개")
print(f"범주형 변수: {len(categorical_cols)}개")
print(f"전체 결측치: {df.isnull().sum().sum():,}개 ({(df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%)")
print(f"중복 행: {df.duplicated().sum():,}개")
print("="*50)